### L1正则

In [1]:
import numpy as np 

#定义符号函数
def sign(w):
    if w > 0:
        return 1
    elif w == 0:
        return 0
    else:
        return -1
#利用numpy对符号函数进行向量化，就是一次可以处理多个w，比如w1, w2, w3……
vec_sign = np.vectorize(sign)

In [2]:
'''
关于np.vectorize用法的例子
def myfunc(a, b):
...     "Return a-b if a>b, otherwise return a+b"
...     if a > b:
...         return a - b
...     else:
...         return a + b
    

vfunc = np.vectorize(myfunc)

vfunc([1, 2, 3, 4], 2)
Out[87]: array([3, 4, 1, 2])
'''

'\n关于np.vectorize用法的例子\ndef myfunc(a, b):\n...     "Return a-b if a>b, otherwise return a+b"\n...     if a > b:\n...         return a - b\n...     else:\n...         return a + b\n    \n\nvfunc = np.vectorize(myfunc)\n\nvfunc([1, 2, 3, 4], 2)\nOut[87]: array([3, 4, 1, 2])\n'

In [3]:
w = np.array([[1],[-3]])
vec_sign = np.vectorize(sign)
vec_sign(w)

array([[ 1],
       [-1]])

In [4]:
#初始化参数
def initialize_param(dims):
    w = np.zeros((dims,1))
    b = 0
    return w, b

In [5]:
#定义L1正则化的线性回归
def linear_regression_L1(X, y, w, b, alpha):
    num_train = X.shape[0]
    y_hat = np.dot(X, w) + b
    loss = np.sum((y_hat-y)**2)/num_train + np.sum(alpha*abs(w))
    
    dw = np.dot(X.T, (y_hat-y))/num_train + alpha*vec_sign(w) #因为w可能是多维的，所以不能直接用sign函数
    db = db = np.sum((y_hat-y)) /num_train
    return y_hat, loss, dw, db

In [6]:
def training_process_L1(X, y, alpha=0.5, learning_rate=0.001, epoch=500):
    
    w, b = initialize_param(X.shape[1])
    
    loss_his = []
    for i in range(epoch):
        yhat, loss, dw, db = linear_regression_L1(X, y, w, b, alpha)
        
        w += -learning_rate * dw
        b += -learning_rate * db
        
        loss_his.append(loss)
        
        if i%50 == 0:
            print('epoch %d loss %f'%(i, loss))
        params = {
            'w' : w,
            'b' : b            
        }
        
        grads = {
            'dw' : dw,
            'db' : db
        }
    return loss_his, params, grads

In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd
import warnings
import numpy as np
# 加载波士顿房屋数据集
data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
feature_name = ['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','MEDV']
# 数据准备
X = data
y = target

#dataset = load_boston()
df = pd.DataFrame(X,columns=feature_name)
df['price'] = y
print(df.head())

X = df[['CRIM','RM']].values
y = df['price'].values

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\BDY\AppData\Local\Temp\ipykernel_17484\1297280390.py:7: SyntaxWarning: invalid escape sequence '\s'
  raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)


      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  MEDV  price  
0     15.3  396.90  4.98   24.0  
1     17.8  396.90  9.14   21.6  
2     17.8  392.83  4.03   34.7  
3     18.7  394.63  2.94   33.4  
4     18.7  396.90  5.33   36.2  


In [8]:
X.shape

(506, 2)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.2, shuffle=True)

In [10]:
y_train = y_train.reshape((-1,1))
y_test = y_test.reshape((-1,1))

In [14]:
loss_his, params, grads = training_process_L1(X_train, y_train, alpha=0.3, learning_rate=0.01, epoch=500)
print(params)

epoch 0 loss 588.034356
epoch 50 loss 48.672657
epoch 100 loss 48.601641
epoch 150 loss 48.531364
epoch 200 loss 48.461816
epoch 250 loss 48.392991
epoch 300 loss 48.324881
epoch 350 loss 48.257478
epoch 400 loss 48.190775
epoch 450 loss 48.124764
{'w': array([[-0.33084847],
       [ 3.93101223]]), 'b': -0.6562129273271913}


### L2正则

In [15]:
#整体架构和L1正则基本一致，唯独在损失函数和更新梯度的部分进行调整
#定义L2正则化的线性回归
def linear_regression_L2(X, y, w, b, alpha):
    num_train = X.shape[0]
    y_hat = np.dot(X, w) + b
    loss = np.sum((y_hat-y)**2)/num_train + alpha*np.sum(np.square(w))
    
    dw = np.dot(X.T, (y_hat-y))/num_train + 2*alpha*w 
    db = db = np.sum((y_hat-y)) /num_train
    return y_hat, loss, dw, db

In [16]:
def training_process_L2(X, y, alpha=0.5, learning_rate=0.001, epoch=500):
    
    w, b = initialize_param(X.shape[1])
    
    loss_his = []
    for i in range(epoch):
        #选择L2正则
        yhat, loss, dw, db = linear_regression_L2(X, y, w, b, alpha)
        
        w += -learning_rate * dw
        b += -learning_rate * db
        
        loss_his.append(loss)
        
        if i%50 == 0:
            print('epoch %d loss %f'%(i, loss))
        params = {
            'w' : w,
            'b' : b            
        }
        
        grads = {
            'dw' : dw,
            'db' : db
        }
    return loss_his, params, grads

In [17]:
loss_his, params, grads = training_process_L2(X_train, y_train, alpha=0.3, learning_rate=0.01, epoch=500)
print(params)

epoch 0 loss 588.034356
epoch 50 loss 51.768135
epoch 100 loss 51.772964
epoch 150 loss 51.777745
epoch 200 loss 51.782478
epoch 250 loss 51.787165
epoch 300 loss 51.791805
epoch 350 loss 51.796398
epoch 400 loss 51.800945
epoch 450 loss 51.805447
{'w': array([[-0.32428257],
       [ 3.66066917]]), 'b': 0.7403535862434624}


### 书中数据集的正则化回归

In [18]:
import numpy as np
data = np.genfromtxt('example.dat', delimiter=',')
print(data)
data.shape

[[-1.14558 -1.29249  0.84911 ...  1.5532  -1.42135  1.19238]
 [ 1.38724 -1.00201 -0.3337  ...  0.81903  0.39286 -3.44094]
 [ 1.47233  0.8488  -0.33866 ...  0.08911 -1.72476  3.75006]
 ...
 [-0.83673  0.80514  0.00807 ... -1.64165  2.04662  1.84121]
 [ 1.12062  0.68561 -1.08    ...  1.1926   0.33696  3.53143]
 [-0.28943 -0.22213 -0.52226 ...  0.22531  1.72576 -0.55118]]


(101, 101)

In [19]:
X = data[:,0:100]
y = data[:,100].reshape((-1,1))
X_train, X_test, y_train, y_test = X[:70], X[70:], y[:70], y[70:]

In [20]:
loss_his_l1, params_l1, grads_l1 = training_process_L1(X_train, y_train, alpha=0.3, learning_rate=0.01, epoch=500)
loss_his_l2, params_l2, grads_l2 = training_process_L2(X_train, y_train, alpha=0.3, learning_rate=0.01, epoch=500)

epoch 0 loss 9.663744
epoch 50 loss 5.304081
epoch 100 loss 4.496554
epoch 150 loss 4.194744
epoch 200 loss 4.036807
epoch 250 loss 3.953707
epoch 300 loss 3.888901
epoch 350 loss 3.847023
epoch 400 loss 3.815737
epoch 450 loss 3.789598
epoch 0 loss 9.663744
epoch 50 loss 2.605747
epoch 100 loss 1.991596
epoch 150 loss 1.837324
epoch 200 loss 1.779207
epoch 250 loss 1.752703
epoch 300 loss 1.739193
epoch 350 loss 1.731787
epoch 400 loss 1.727510
epoch 450 loss 1.724937


In [21]:
np.set_printoptions(suppress=True)
#l1的参数包含大量的0  
np.around(params_l1['w'],2)[:5]

array([[-0.  ],
       [ 0.25],
       [ 0.33],
       [ 0.  ],
       [ 0.75]])

In [22]:
np.around(params_l2['w'],2)[:5]

array([[-0.07],
       [ 0.29],
       [ 0.22],
       [ 0.14],
       [ 0.48]])